In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import utilix
import numpy as np
from tqdm import tqdm
import os
from tqdm import tqdm
path_start = '/dali/lgrandi/rucio/xnt_'
import gc
import pickle
from utilix import xent_collection
from glob import glob
import sys
sys.path.append('/home/yuanlq/software/xeda')
from xeda import utilities

In [ ]:
coll = xent_collection()
sr0_right = 34731
sr0_left  = 17918
sr1_left  = 43039
sr1_right = coll.count_documents({})

In [ ]:
with open('/project2/lgrandi/xenonnt/reprocessing_runlist/global_v11/runlists_reprocessing_global_v11.pickle', 'rb') as f:
    jingqiang = pickle.load(f)

In [ ]:
tpc_runlist = np.array([])
nv_runlist = np.array([])

for rnlist in jingqiang['runlists'].values():
    tpc_runlist = np.concatenate((tpc_runlist, rnlist))
for mode in ['th232', 'sr1_ybe', 'sr1_ambe', 'sr1_bkg']:
    nv_runlist = np.concatenate((nv_runlist, jingqiang['runlists'][mode]))

In [ ]:
rules_info = utilities.load_rules(datestr=20230710)

In [ ]:
peaklets = rules_info[rules_info['data_type']=='peaklets']
merged_s2s = rules_info[rules_info['data_type']=='merged_s2s']
lone_hits = rules_info[rules_info['data_type']=='lone_hits']
hitlets_nv = rules_info[rules_info['data_type']=='hitlets_nv']
del rules_info
gc.collect()

In [ ]:
peaklets = peaklets[peaklets['UC_DALI_USERDISK']]
merged_s2s = merged_s2s[merged_s2s['UC_DALI_USERDISK']]
lone_hits = lone_hits[lone_hits['UC_DALI_USERDISK']]
hitlets_nv = hitlets_nv[hitlets_nv['UC_DALI_USERDISK']]

In [ ]:
peaklets_v11 = utilities.in_runlist(peaklets, tpc_runlist)
merged_s2s_v11 = utilities.in_runlist(merged_s2s, tpc_runlist)
lone_hits_v11 = utilities.in_runlist(lone_hits, tpc_runlist)
hitlets_nv_v11 = utilities.in_runlist(hitlets_nv, nv_runlist)

In [ ]:
cum_sizes_tb_pkl = utilities.size_vs_runs(peaklets)
cum_sizes_tb_hnv = utilities.size_vs_runs(hitlets_nv)
cum_sizes_tb_ms2 = utilities.size_vs_runs(merged_s2s)
cum_sizes_tb_lhs = utilities.size_vs_runs(lone_hits)

In [ ]:
cum_sizes_tb_pkl_v11 = utilities.size_vs_runs(peaklets_v11)
cum_sizes_tb_hnv_v11 = utilities.size_vs_runs(hitlets_nv_v11)
cum_sizes_tb_ms2_v11 = utilities.size_vs_runs(merged_s2s_v11)
cum_sizes_tb_lhs_v11 = utilities.size_vs_runs(lone_hits_v11)

In [ ]:
utilities.plot_cum_sizes_tb(title='Peaks on DaLI', x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets=peaklets,
                            merged_s2s=merged_s2s,
                            lone_hits=lone_hits,
                            hitlets_nv=hitlets_nv,
                            peaklets_v11=peaklets_v11,
                            merged_s2s_v11=merged_s2s_v11,
                            lone_hits_v11=lone_hits_v11,
                            hitlets_nv_v11=hitlets_nv_v11)

In [ ]:
np.sum(cum_sizes_tb_pkl[-1]+cum_sizes_tb_ms2[-1]+cum_sizes_tb_hnv[-1]+cum_sizes_tb_lhs[-1])

In [ ]:
np.sum(cum_sizes_tb_pkl_v11[-1]+cum_sizes_tb_ms2_v11[-1]+cum_sizes_tb_hnv_v11[-1]+cum_sizes_tb_lhs_v11[-1])

# Rules NOT in v11

In [ ]:
peaklets_diff = np.setdiff1d(peaklets, peaklets_v11)
merged_s2s_diff = np.setdiff1d(merged_s2s, merged_s2s_v11)
lone_hits_diff = np.setdiff1d(lone_hits, lone_hits_v11)
hitlets_nv_diff = np.setdiff1d(hitlets_nv, hitlets_nv_v11)

In [ ]:
cum_sizes_tb_pkl_diff = utilities.size_vs_runs(peaklets_diff)
cum_sizes_tb_hnv_diff = utilities.size_vs_runs(hitlets_nv_diff)
cum_sizes_tb_ms2_diff = utilities.size_vs_runs(merged_s2s_diff)
cum_sizes_tb_lhs_diff = utilities.size_vs_runs(lone_hits_diff)

In [ ]:
utilities.plot_cum_sizes_tb(title='Peak-Level Data on DaLI but runs NOT in v11 processing list', 
                            x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets_diff=peaklets_diff,
                            merged_s2s_diff=merged_s2s_diff,
                            lone_hits_diff=lone_hits_diff,
                            hitlets_nv_diff=hitlets_nv_diff)

In [ ]:
np.sum(cum_sizes_tb_pkl_diff[-1]+cum_sizes_tb_ms2_diff[-1]+cum_sizes_tb_hnv_diff[-1]+cum_sizes_tb_lhs_diff[-1])

What are the run modes for them?

In [ ]:
peaklets_diff_unique_modes, peaklets_diff_sizes_tb = utilities.size_vs_mode(peaklets_diff, 
                                                                            title='peaklets Data on DaLI but runs NOT in v11 processing list')
merged_s2s_unique_modes, merged_s2s_diff_sizes_tb = utilities.size_vs_mode(merged_s2s_diff, 
                                                                            title='merged_s2s Data on DaLI but runs NOT in v11 processing list')
lone_hits_diff_unique_modes, lone_hits_diff_sizes_tb = utilities.size_vs_mode(lone_hits_diff, 
                                                                            title='lone_hits Data on DaLI but runs NOT in v11 processing list')
hitlets_nv_unique_modes, hitlets_nv_diff_sizes_tb = utilities.size_vs_mode(hitlets_nv_diff, 
                                                                            title='hitlets_nv Data on DaLI but runs NOT in v11 processing list')

OK let's only check the sensible range

In [ ]:
import importlib
importlib.reload(utilities)

In [ ]:
peaklets_diff_unique_modes, peaklets_diff_sizes_tb = utilities.size_vs_mode(peaklets_diff[peaklets_diff['runid'].astype(int)>sr0_left], 
                                                                            title='Since SR0 peaklets Data on DaLI but runs NOT in v11 processing list')
merged_s2s_unique_modes, merged_s2s_diff_sizes_tb = utilities.size_vs_mode(merged_s2s_diff[merged_s2s_diff['runid'].astype(int)>sr0_left], 
                                                                            title='Since SR0 merged_s2s Data on DaLI but runs NOT in v11 processing list')
lone_hits_diff_unique_modes, lone_hits_diff_sizes_tb = utilities.size_vs_mode(lone_hits_diff, 
                                                                            title='All lone_hits Data on DaLI but runs NOT in v11 processing list')
hitlets_nv_unique_modes, hitlets_nv_diff_sizes_tb = utilities.size_vs_mode(hitlets_nv_diff[hitlets_nv_diff['runid'].astype(int)>sr1_left], 
                                                                            title='Since SR1 hitlets_nv Data on DaLI but runs NOT in v11 processing list')

# All SR1 Peaks in v11 TPC list

In [ ]:
rules_info = utilities.load_rules(datestr=20230710)
peaklets = rules_info[rules_info['data_type']=='peaklets']
merged_s2s = rules_info[rules_info['data_type']=='merged_s2s']
lone_hits = rules_info[rules_info['data_type']=='lone_hits']
hitlets_nv = rules_info[rules_info['data_type']=='hitlets_nv']
del rules_info
gc.collect()

peaklets = peaklets[peaklets['UC_DALI_USERDISK']]
merged_s2s = merged_s2s[merged_s2s['UC_DALI_USERDISK']]
lone_hits = lone_hits[lone_hits['UC_DALI_USERDISK']]
hitlets_nv = hitlets_nv[hitlets_nv['UC_DALI_USERDISK']]

In [ ]:
peaklets_sr1_v11 = utilities.in_runlist(peaklets, tpc_runlist[tpc_runlist>sr1_left])
merged_s2s_sr1_v11 = utilities.in_runlist(merged_s2s, tpc_runlist[tpc_runlist>sr1_left])
lone_hits_sr1_v11 = utilities.in_runlist(lone_hits, tpc_runlist[tpc_runlist>sr1_left])

In [ ]:
peaklets_sr1_v11['size_gb'].sum()/1024

In [ ]:
merged_s2s_sr1_v11['size_gb'].sum()/1024

In [ ]:
lone_hits_sr1_v11['size_gb'].sum()/1024

# Proposal for deletion

In [ ]:
# SR1 hitlets_nv not in v11 reprocessing list
sr1_dali_hitlets_nv_not_v11list = hitlets_nv_diff[hitlets_nv_diff['runid'].astype(int)>sr1_left]
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/sr1_dali_hitlets_nv_not_v11list.npy', 
        sr1_dali_hitlets_nv_not_v11list)

# Since SR0 peaklets/merged_s2s not in v11 reprocessing list
since_sr0_dali_peaklets_not_v11list = peaklets_diff[peaklets_diff['runid'].astype(int)>sr0_left]
since_sr0_dali_merged_s2s_not_v11list = merged_s2s_diff[merged_s2s_diff['runid'].astype(int)>sr0_left]
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/since_sr0_dali_peaklets_not_v11list.npy', 
        since_sr0_dali_peaklets_not_v11list)
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/since_sr0_dali_merged_s2s_not_v11list.npy', 
        since_sr0_dali_merged_s2s_not_v11list)

# All lonehits not in v11 reprocessing list
all_lonehits_not_v11list = lone_hits_diff
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/all_lonehits_not_v11list.npy', 
        all_lonehits_not_v11list)

In [ ]:
np.sum(sr1_dali_hitlets_nv_not_v11list['size_gb'])/1024

In [ ]:
np.sum(since_sr0_dali_peaklets_not_v11list['size_gb'])/1024

In [ ]:
np.sum(since_sr0_dali_merged_s2s_not_v11list['size_gb'])/1024

In [ ]:
np.sum(all_lonehits_not_v11list['size_gb'])/1024

In [ ]:
# SR1 peaklets in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/peaklets_sr1_v11.npy', peaklets_sr1_v11)

# SR1 merged_s2s in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/merged_s2s_sr1_v11.npy', merged_s2s_sr1_v11)

# SR1 lone_hits in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/lone_hits_sr1_v11.npy', lone_hits_sr1_v11)